In [ ]:
import cv2 as cv
import numpy as np
from ultralytics import YOLO
from matplotlib import pyplot as plt
from scipy.spatial.distance import euclidean
import math

In [ ]:
# Credit: ChatGPT
def perpendicular_distance(point, start, end):
    """Calculate the perpendicular distance from a point to a line segment."""
    if np.all(start == end):
        return euclidean(point, start)
    
    line_vec = end - start
    point_vec = point - start
    line_len = np.dot(line_vec, line_vec)
    t = max(0, min(1, np.dot(point_vec, line_vec) / line_len))
    projection = start + t * line_vec
    return euclidean(point, projection)

# Credit: ChatGPT
def rdp(contour, epsilon):
    """
    Applies the Ramer-Douglas-Peucker (RDP) algorithm to simplify a contour.
    
    Parameters:
        contour (np.ndarray): A Nx2 array of (x, y) coordinates.
        epsilon (float): Tolerance for point reduction (higher -> more aggressive).
    
    Returns:
        np.ndarray: Simplified contour as a Nx2 array.
    """
    if len(contour) < 3:
        return contour

    # Find the point with the maximum perpendicular distance
    start, end = contour[0], contour[-1]
    distances = np.array([perpendicular_distance(p, start, end) for p in contour[1:-1]])
    
    max_idx = np.argmax(distances)
    max_dist = distances[max_idx]
    
    if max_dist > epsilon:
        max_idx += 1  # Offset for skipping the first point
        # Recursive RDP on both segments
        left = rdp(contour[:max_idx+1], epsilon)
        right = rdp(contour[max_idx:], epsilon)
        return np.vstack((left[:-1], right))
    else:
        return np.array([start, end])

In [ ]:
def simplify(contour):
    points = contour.shape[0]
    if points == 4:
        return contour
    full_area = cv.contourArea(contour)
    d_min = math.inf
    point_min = None
    for point in range(points):
        area = cv.contourArea(np.delete(contour, point, axis=0))
        if (d := abs(full_area - area)) <= d_min:
            d_min = d
            point_min = point
    if point_min is None:
        raise ValueError("That's cool I guess.")
    return simplify(np.delete(contour, point_min, axis=0))

def warp(src, src_quad, dst_quad):
    h, _ = cv.findHomography(dst_quad, src_quad)
    _, _, w_dst, h_dst = cv.boundingRect(dst_quad)
    return cv.warpPerspective(src, h, (w_dst, h_dst), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP)

def normal_to_image(image_shape, contour):
    h, w, _ = image_shape
    return (contour * (w, h)).astype(int)

In [ ]:
image = "datasets/coco2017/val/images/000000195918.jpg"
result = YOLO("runs/segment/filtered-5e/weights/best.pt")(image)[0]
masks = YOLO("runs/segment/filtered-5e/weights/best.pt")(image)[0].masks.xyn
image = cv.imread(image)
contours = [
    normal_to_image(image.shape, simplify(rdp(mask, 0.01))) for mask in masks
]

for i, contour in enumerate(contours):
    image = cv.polylines(image, [contour], True, (255, 0, 0))
    image = cv.putText(image, str(i), contour[0], cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
plt.imshow(image)

In [ ]:
dst = np.array([[0, 0], [0, 300], [500, 300], [500, 0]])
plt.imshow(warp(image, contours[2], dst))